In [11]:
import json
import requests
import logging, os, re, datetime
from IPython.display import display
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.mapping import WebScene
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
from arcgis.features import FeatureSet

import time

In [12]:
SaveLogsTo = '../Logs'
logging.basicConfig(level=logging.INFO)
logging.getLogger('arcgis.gis._impl').setLevel(logging.WARNING)
logger = logging.getLogger(__name__)

logFileName = datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
fileHandler = logging.handlers.RotatingFileHandler('{}/{}.log'.format(SaveLogsTo, logFileName), maxBytes=100000, backupCount=5)
formatter = logging.Formatter('%(asctime)s %(levelname)s %(relativeCreated)d \n%(filename)s %(module)s %(funcName)s %(lineno)d \n%(message)s\n')
fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)
logger.info('Script Starting at {}'.format(str(datetime.datetime.now())))

INFO:__main__:Script Starting at 2024-02-23 13:16:47.968413


# MIGRATION

## Initialise Migration

In [13]:
# imports
from classes.contentmigration import GisMigration, MigrateUser
import scripts.portalmanager as pm

In [14]:
# Define parameters for this Migration
portals_csv = '../Conf/portal_connect_deets.csv'
MIGRATE_DATA_FOLDER = '../migration_data'
usr_args_migration_name = "ISP_to_DIGITAL_DEV"  # "ISP_TO_UAT"
usr_args_from_gis_alias_name = 'ISP_OAUTH2'
usr_args_to_gis_alias_name = 'DIGITAL_DEV'  # 'ISP'
usr_args_from_usernames = ['Santiago.patino@aurecongroup.com', 'Katleya.DelaCruz@aurecongroup.com']  # '*'
usr_args_select_itemids = None
usr_args_select_types = None #['form']

#  usr_args_usernames = '*'

migration_username = 'Santiago.patino@aurecongroup.com'

In [15]:
today = datetime.datetime.now().strftime('%d%m%Y')
data_folder = '../migration_data'
fromgis = pm.portal_connect(portals_csv, usr_args_from_gis_alias_name,
                            username=None,
                            auth_type='OAuth 2.0')
if usr_args_to_gis_alias_name:
    togis = pm.portal_connect(portals_csv, usr_args_to_gis_alias_name,
                              username=None,
                              auth_type='Pro')
                             

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ispgeospatial.com/portal/sharing/rest/oauth2/authorize?response_type=code&client_id=K7qStPWY3MPp0b8z&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=iytXKWMJqveurXIQbfXsIsKs5n5cAV&allow_verification=false
Enter code obtained on signing in using SAML: ········


C:\Users\santiago.patino\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ispgeospatial.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Successfully logged in GIS https://ispgeospatial.com/portal as: Santiago.Patino@aurecongroup.com
Successfully logged in GIS https://ppgeospatial-dev.aurecongroup.digital/arcgis/ as: Santiago.Patino@aurecongroup.com


In [6]:
gis_migration = GisMigration(today,
                             usr_args_migration_name,
                             fromgis,
                             togis,
                             MIGRATE_DATA_FOLDER,
                             migrate_usernames=usr_args_from_usernames,
                             migrate_item_ids=usr_args_select_itemids,
                             migrate_item_types=usr_args_select_types
                             )
logger.info(f'Migration {gis_migration.name} initialised at {str(datetime.datetime.now())}')

Connected to GIS @ https://ispgeospatial.com/portal version:10.3


INFO:__main__:Migration ISP_to_DIGITAL_DEV initialised at 2024-02-23 12:00:50.469975


In [10]:
username = ['Katleya.DelaCruz@aurecongroup.com']
gis_migration.migrate_users = gis_migration.get_migrate_users(usernames=username)
user = gis_migration.migrate_users[0]
print(f"Migrating content for users: {user}")


migrate_user = MigrateUser(gis_migration, user)
logger.info(f'MigrateUser {migrate_user} initialised at {str(datetime.datetime.now())}')
migrate_user.migrate_items = migrate_user.get_migrate_user_items_list(itemids_list=gis_migration.item_ids,
                                                      item_types=gis_migration.item_types)

user_content_df = migrate_user.make_migrate_items_df()
print(user_content_df.shape)
# user_content_df = user_content_df.set_index(['item_id', 'dependentupon_id'])
# user_content_df
# show_df = user_content_df['item_id','dependentupon_id']


# Display the DataFrame
# show_df
# # Get the columns of the DataFrame
# columns = user_content_df.columns
# 
# # Display the columns
# print(columns)

Connected to GIS @ https://ispgeospatial.com/portal version:10.3
Migrating content for users: <User username:Katleya.DelaCruz@aurecongroup.com>


INFO:__main__:MigrateUser Katleya.DelaCruz@aurecongroup.com initialised at 2024-02-23 12:11:46.271344


("unhashable type: 'User'",)


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# FOR ALL - uncomment to go through all users in a for loop

# gis_migration.migrate_users = gis_migration.get_migrate_users(usernames=usr_args_from_usernames)
# print(f"Migrating content for users: {gis_migration.migrate_users}")
# for user in gis_migration.migrate_users:
#     print(f"user: {user}")
#     migrate_user = MigrateUser(gis_migration, user)
#     logger.info(f'MigrateUser {migrate_user} initialised at {str(datetime.datetime.now())}')
#     migrate_user.migrate_items = migrate_user.get_migrate_user_items_list(itemids_list=gis_migration.item_ids,
#                                                           item_types=gis_migration.item_types)
#     print(f"User {user} has {len(migrate_user.migrate_items)} items to migrate")
#     user_content_df = migrate_user.make_migrate_items_df()
#     import scripts.gp_functions as gp
#     output_csv = gp.df_export_to_csv(migrate_user.user.username, user_content_df, gis_migration.data_folder, today)


## Process Survey123 items for Users in this Migration

In [7]:
## Process Survey123 items for a user in this Migration
import arcgis
from arcgis.gis import GIS
import os

# get the items of type:form
survey_manager = arcgis.apps.survey123.SurveyManager(gis_migration.migrate_from)
migrate_user.migrate_items

NameError: name 'migrate_user' is not defined

In [0]:


group = gis_migration.migrate_from.groups.get('0dd43178863545fbb5526278e1bdfc4c')
sourceForms = group.search('type:form')['results']
sourceForms
    

In [0]:
## recall code ;))from arcgis.gis import GIS

# # Connect to the source and destination portals
# source_portal_url = "https://source_portal_url/arcgis"
# destination_portal_url = "https://destination_portal_url/arcgis"
# source_portal = GIS(source_portal_url, "username", "password")
# destination_portal = GIS(destination_portal_url, "username", "password")
# 
# # Define the form item ID
# form_item_id = "form_item_id"
# 
# def find_related_items(form_item_id, portal):
#     related_items = []
# 
#     # Get the item related to the form
#     form_item = portal.content.get(form_item_id)
# 
#     # Get items related to the form
#     related_items.extend(form_item.related_items("Dependent"))
# 
#     # Get items that the form is dependent on
#     related_items.extend(form_item.related_items("DependsOn"))
# 
#     return related_items
# 
# def clone_and_migrate_items(form_item_id, source_portal, destination_portal):
#     # Find all related items
#     related_items = find_related_items(form_item_id, source_portal)
#     related_item_ids = [item.itemid for item in related_items]
# 
#     # Clone and migrate each related item
#     for item_id in related_item_ids:
#         source_item = source_portal.content.get(item_id)
#         destination_portal.content.clone_items([source_item], folder="target_folder", copy_data=False)
# 
# # Clone and migrate all related items
# clone_and_migrate_items(form_item_id, source_portal, destination_portal)



In [ ]:
# Get surveys by group ID and then download each format supported
# survey_manager = arcgis.apps.survey123.SurveyManager(source_gis)
# group = source.groups.get('0dd43178863545fbb5526278e1bdfc4c')
# sourceForms = group.search('type:form')['results']
# sourceForms

